1. OCR을 활용하여 구현하려는 서비스의 기획이 타당한가?

- 목표로 하는 서비스가 OCR를 적용 가능하며, OCR을 활용했을 때 더욱 유용해진다.

2. 모델 평가기준이 명확하고 체계적으로 세워졌는가?

- 평가 기준에 부합하는 테스트 데이터의 특징이 무엇인지 명확하게 제시되었다.

3. 평가기준에 따라 충분한 분량의 테스트가 진행되고 그 결과가 잘 정리되었는가?

- 최대 20장까지의 테스트 이미지를 사용해 제시된 평가 기준에 따른 테스트 결과가 잘 정리되어 결론이 도출되었다.

# Step1. 검증용 데이터셋 준비
OCR 관련해서 오늘 다루어본 주제는 텍스트의 모양과 크기, 배치, 노이즈 등 OCR의 성능에 영향을 미치는 요인들에 관한 것이었습니다. 여러분들은 위 3가지 기능들이 이미지에 나타난 특징에 따라 얼마나 정확하게 문자를 detect하고 recognize하는지를 검증하기 위한 이미지 데이터를 모아야 합니다. 이미지가 많을수록 좋겠지만 검증해야 할 항목별로 고루 갖춰지는 것도 중요합니다.

가능하다면 함께 학습하는 동료들과 힘을 합쳐보는 것을 추천합니다. 그렇다고 너무 많은 이미지 데이터로 검증하는 것은 이후 결과를 정리하는 것이 힘들 수 있으니 테스트용 이미지는 아무리 많아도 20장을 넘기지 않는 것으로 합시다.


-------------------------------------

손글씨를 인식하여 
1. 맹인이나 눈이 안 좋은 이들, 문맹자에게 손글씨 편지를 음성으로 읽어주어 편지를 읽어줄 수 있다.
2. 키보드에 익숙하지 않은, 디지털 역량이 떨어지는 자들의 손글씨로 쓴 글을 디지털로 문서화할 수 있다.
3. 학생들이 필기한 노트를 인식, 업로드하여 쉽게 파일로 저장할 수 있다.


# Step2. Google OCR API, keras-ocr, Tesseract로 테스트 진행
위 3가지 모듈을 활용하는 방법에 대해서는 코드 레벨로 이미 경험해 보셨을 것입니다. 이미지 리스트를 정리한 후 반복문을 실행하며 모듈이 출력한 결과를 모아 봅시다.

# 1. Google OCR API

In [1]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()
        
    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print('Texts:')

    for text in texts:
       print('\n"{}"'.format(text.description))

    vertices = (['({},{})'.format(vertex.x, vertex.y)
                 for vertex in text.bounding_poly.vertices])

    print('bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [10]:
# 다운받은 인증키 경로가 정확하게 지정되어 있어야 합니다. 
!ls -l $GOOGLE_APPLICATION_CREDENTIALS

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] =  os.getenv('HOME')+'/aiffel/ocr_python/my_google_api_key.json'

# for문이 왜 안돌까? 
'''
# 입력 이미지 경로를 지정해 주세요.
for image in range(1,3):
    path = os.getenv('HOME')+'/aiffel/ocr_python/handwriting_kor'
    f'path/{image}.*'
    detect_text(image)
'''
image = os.getenv('HOME')+'/aiffel/ocr_python/handwriting_kor/3.jpeg'
detect_text(image)
# path = # [[YOUR IMAGE FILE PATH]]   

# 위에서 정의한 OCR API 이용 함수를 호출해 봅시다.
    

-rw-rw-r-- 1 aiffel aiffel 2325 Mar 18 16:43 /home/aiffel/aiffel/ocr_python/my_google_api_key.json
Texts:

"TOOLS toLIVEBY
OSUN OMON OTUE OWED CTHU O
보 한바가지 부어서
한길 당신 속까지 바중 갔다가
않께 취석이는 거객요
올라온 물과 서이병
그 힘 바가지의 역
이름을 볼여수어
철렁하기도 해가
참 어이없게도
"

"TOOLS"

"toLIVEBY"

"OSUN"

"OMON"

"OTUE"

"OWED"

"CTHU"

"O"

"보"

"한바"

"가지"

"부어서"

"한길"

"당신"

"속"

"까지"

"바중"

"갔다가"

"않께"

"취"

"석이"

"는"

"거"

"객요"

"올라온"

"물"

"과"

"서"

"이병"

"그"

"힘"

"바가지"

"의"

"역"

"이름"

"을"

"볼"

"여수"

"어"

"철렁"

"하기도"

"해"

"가"

"참"

"어이"

"없게도"
bounds: (296,1129),(357,1088),(486,1282),(426,1323)


# 2. Keras-ocr

In [11]:
import matplotlib.pyplot as plt
import keras_ocr

# keras-ocr이 detector과 recognizer를 위한 모델을 자동으로 다운로드받게 됩니다. 
pipeline = keras_ocr.pipeline.Pipeline()

Looking for /home/aiffel/.keras-ocr/craft_mlt_25k.h5
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Looking for /home/aiffel/.keras-ocr/crnn_kurapan.h5


In [ ]:
# 테스트에 사용할 이미지 url을 모아 봅니다. 추가로 더 모아볼 수도 있습니다. 
image_urls = [
  'https://unsplash.com/photos/AXnLoubqSOo',
  'https://source.unsplash.com/6jsp4iHc8hI/640x460',
  'https://source.unsplash.com/98uYQ-KupiE',
  'https://source.unsplash.com/j9JoYpaJH3A',
  'https://source.unsplash.com/eBkEJ9cH5b4'
]

images = [ keras_ocr.tools.read(url) for url in image_urls]
prediction_groups = [pipeline.recognize([url]) for url in image_urls]

In [13]:
img_ker = [os.getenv('HOME')+'/aiffel/ocr_python/handwriting_kor/1.jpg',
    os.getenv('HOME')+'/aiffel/ocr_python/handwriting_kor/2.jpg',
    os.getenv('HOME')+'/aiffel/ocr_python/handwriting_kor/4.jpeg',
    os.getenv('HOME')+'/aiffel/ocr_python/handwriting_kor/5.jpeg'
    
]

imgs = [keras_ocr.tools.read(img) for img in img_ker]
pred = [pipeline.recognize([img])for img in img_ker]

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model_1/basenet.slice1.0/Conv2D (defined at /home/aiffel/anaconda3/envs/aiffel/lib/python3.7/site-packages/keras_ocr/detection.py:682) ]] [Op:__inference_predict_function_6881]

Function call stack:
predict_function


# 3. Tesseract

# Step3. 테스트 결과 정리
모듈들마다 출력 형태가 다릅니다. 분석을 위해 결과를 일목요연하게 정리해 보는 것이 도움이 될 것입니다. 시각화 방법이 함께 활용된다면 더욱 좋겠군요.

# Step4. 결과 분석과 결론 제시
- 우선 여러분들이 구현하고자 하는 서비스의 목적과 내용을 먼저 정리합니다.
- 여러분들이 검토한 OCR 모델들이 그 목적에 부합하는지 확인하기 위해 필요한 평가기준을 제시합니다.
- 그 기준에 따라 위에서 정리한 테스트 결과를 분석하여 가장 나은 대안의 모델을 선정합니다.